<a href="https://colab.research.google.com/github/falng2/AI-660710711/blob/main/RAGLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-openai langchain-ollama langchain-community faiss-cpu duckdb-engine wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=7e6e456dfd5f002e0e4ec7357944617ea2a264eac6d1cc09f66de8ceaebeed8d
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver doe

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [4]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [5]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    


In [6]:
%%time
!ollama run scb10x/typhoon2.1-gemma3-4b:latest "สวัสดี"

สวัสดีครับ! ยินดีที่ได้รู้จักครับ ผมชื่อ Typhoon สร้างโดย SCB 10X ครับ ผมพร้อมที่จะช่วยเหลือคุณในทุกเรื่องที่ผมสามารถทำได้ครับ มีอะไรให้ผมช่วยไหมครับ?

CPU times: user 25.4 ms, sys: 5.86 ms, total: 31.2 ms
Wall time: 5.05 s


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
!ls /content/drive/MyDrive/RAG

MCU


In [41]:
from langchain_ollama import OllamaLLM

In [42]:
llm = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [43]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีครับ! ยินดีที่ได้รู้จักครับ ผม Typhoon ซึ่งเป็น AI Assistant ที่สร้างโดย SCB 10X ครับ มีอะไรให้ผมช่วยครับ? 😊



In [44]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_ollama import ChatOllama

In [11]:
llm2 = ChatOllama(model="scb10x/typhoon2.1-gemma3-4b:latest")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm2, memory=memory, verbose=False)
conversation.predict(input="สวัสดีครับ")
conversation.predict(input="จำได้มั้ยว่าผมทักว่าอะไร?")

'I’m sorry, I don’t have access to past conversations. I don’t retain any memory of previous interactions. I’m designed to treat each conversation as a fresh start. \n\nIs there anything else I can help you with?'

In [12]:
conversation.predict(input="สวัสดีครับ")
conversation.predict(input="จำได้มั้ยว่าผมทักว่าอะไร?")

'I’m sorry, I don’t have access to past conversations. I don’t retain any memory of previous interactions. I’m designed to treat each conversation as a fresh start. \n\nIs there anything else I can help you with?'

In [13]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

In [14]:
docs = [
    Document(page_content="LangChain คือ framework สำหรับ LLM"),
    Document(page_content="Python เป็นภาษายอดนิยมสำหรับ AI")
]

In [45]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [17]:
qa = RetrievalQA.from_chain_type(
    llm=llm2,
    retriever=vectorstore.as_retriever()
)

NameError: name 'vectorstore' is not defined

In [18]:
result = qa.invoke("LangChain คืออะไร")

NameError: name 'qa' is not defined

In [20]:
print(result['result'])

NameError: name 'result' is not defined

In [21]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool

In [22]:
def calculate(expression: str) -> str:
    """คำนวณทางคณิตศาสตร์แบบปลอดภัยขึ้น (จำกัดฟังก์ชัน)"""
    import math
    allowed = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
    allowed.update({"abs": abs, "round": round})
    try:
        return str(eval(expression, {"__builtins__": {}}, allowed))
    except Exception:
        return "ไม่สามารถคำนวณได้"

In [23]:
tools = [
    Tool(
        name="Calculator",
        func=calculate,
        description="ใช้สำหรับคำนวณทางคณิตศาสตร์ เช่น +, -, *, /, และฟังก์ชันพื้นฐานของคณิตศาสตร์"
    )
]

In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "ตอบเป็นภาษาไทยอย่างกระชับเมื่อสรุปผลลัพธ์"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [25]:
agent = create_tool_calling_agent(llm2, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_result = agent_executor.invoke({"input": "345 * 789 เท่ากับเท่าไหร่?"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `345 * 789`


272205345 * 789 เท่ากับ 272205

> Finished chain.


In [26]:
output_string = agent_result['output']

print(output_string)

345 * 789 เท่ากับ 272205


In [27]:
import duckdb
from langchain_community.utilities import SQLDatabase

In [28]:
import warnings
from duckdb_engine import DuckDBEngineWarning

warnings.filterwarnings("ignore", category=DuckDBEngineWarning)

In [31]:
db_path = '/mcu_database.duckdb'

db = SQLDatabase.from_uri(f"duckdb:///{db_path}")

In [32]:
from datetime import date
from sqlalchemy import create_engine

In [33]:
today = date(2025, 8, 11)


engine = create_engine(f"duckdb:///{db_path}")

with engine.begin() as conn:
    conn.exec_driver_sql("""
        create or replace table customers (
            customer_id integer,
            customer_name varchar,
            country varchar,
            segment varchar,
            created_at date,
            is_active boolean
        );
    """)
    conn.exec_driver_sql("""
        insert into customers values
        (1,'Alice','Thailand','retail','2025-08-01', true),
        (2,'Bob','Thailand','retail','2025-08-02', true),
        (3,'Carol','Japan','enterprise','2025-08-03', true),
        (4,'Dan','USA','retail','2025-08-04', true),
        (5,'Eve','Thailand','smb','2025-08-05', true),
        (6,'Frank','Japan','smb','2025-08-06', true),
        (7,'Grace','Vietnam','retail','2025-08-07', true),
        (8,'Heidi','Thailand','enterprise','2025-08-08', true),
        (9,'Ivan','USA','retail','2025-08-09', true),
        (10,'Judy','Thailand','retail','2025-08-10', true);
    """)

    conn.exec_driver_sql("""
        create or replace table orders (
            order_id integer,
            customer_id integer,
            amount double,
            status varchar,
            order_date date
        );
    """)
    conn.exec_driver_sql("""
        insert into orders values
        (101,1,  900,'paid','2025-08-08'),
        (102,2, 1100,'paid','2025-08-09'),
        (103,3,  500,'paid','2025-08-09'),
        (104,4,  700,'paid','2025-08-10'),
        (105,5,  450,'cancelled','2025-08-10'),
        (106,6, 1300,'paid','2025-08-10'),
        (107,7,  650,'paid','2025-08-10'),
        (108,8,  800,'paid','2025-08-11'),
        (109,9,  950,'paid','2025-08-11'),
        (110,10,1000,'paid','2025-08-11');
    """)
print("DuckDB ready.")

DuckDB ready.


In [34]:
tables = db.get_usable_table_names()

print(tables)

[]


In [35]:
from langchain.chains import create_sql_query_chain

In [37]:
sql_chain = create_sql_query_chain(llm, db)

NameError: name 'llm' is not defined

In [ ]:
query = sql_chain.invoke({
    "question": "แสดงจำนวนลูกค้าทั้งหมด"
})

In [ ]:
query

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType

# สร้าง SQL Agent
agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
result = agent_executor.invoke({"input": "แสดงจำนวนลูกค้าทั้งหมด"})
print(result)

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool

sql_agent_tool = Tool(
    name="SQL_Database",
    func=agent_executor.invoke,
    description="Useful for answering questions about the customers database."
)

tools_with_sql = [
    Tool(
        name="Calculator",
        func=calculate,
        description="ใช้สำหรับคำนวณทางคณิตศาสตร์ เช่น +, -, *, /, และฟังก์ชันพื้นฐานของคณิตศาสตร์"
    ),
    sql_agent_tool # เพิ่ม SQL Agent tool เข้าไป
]

# สร้าง Agent ใหม่ที่ใช้ Tools list นี้
new_agent = create_tool_calling_agent(llm2, tools_with_sql, prompt)
new_agent_executor = AgentExecutor(agent=new_agent, tools=tools_with_sql, verbose=True)

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# สร้าง Message History
# เราสามารถใช้ dict เพื่อเก็บ history ของแต่ละ session ได้
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    new_agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history", # กำหนด key สำหรับ chat history
)

# ลองเรียกใช้งานด้วย session_id
session_id = "my-session"

In [ ]:
# คำถามแรก
result1 = with_message_history.invoke(
    {"input": "มีลูกค้าทั้งหมดกี่คนในฐานข้อมูล?"},
    config={"configurable": {"session_id": session_id}}
)
result1

In [ ]:
# คำถามที่สอง (อ้างอิงถึงคำถามแรก)
result2 = with_message_history.invoke(
    {"input": "พวกเขาอยู่ในประเทศอะไรบ้าง?"},
    config={"configurable": {"session_id": session_id}}
)
result2

In [ ]:
# create_tool_calling_agent, AgentExecutor,
from langchain.agents import load_tools
# from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
def calculate(expression: str) -> str:
    """คำนวณทางคณิตศาสตร์แบบปลอดภัยขึ้น (จำกัดฟังก์ชัน)"""
    import math
    allowed = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
    allowed.update({"abs": abs, "round": round})
    try:
        return str(eval(expression, {"__builtins__": {}}, allowed))
    except Exception:
        return "ไม่สามารถคำนวณได้"

calculator_tool = Tool(
    name="Calculator",
    func=calculate,
    description="ใช้สำหรับคำนวณทางคณิตศาสตร์ เช่น +, -, *, /, และฟังก์ชันพื้นฐานของคณิตศาสตร์"
)

sql_agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

sql_agent_tool = Tool(
    name="SQL_Database",
    func=sql_agent_executor.invoke,
    description="Useful for answering questions about the customers database."
)

# โหลด Wikipedia Tool โดยใช้ load_tools
wikipedia_tools = load_tools(["wikipedia"], llm=llm)

# สร้าง Tools list ที่มี Tool ทั้งหมดที่ต้องการให้ Agent ใช้
all_tools = [
    calculator_tool,
    sql_agent_tool,
]
all_tools.extend(wikipedia_tools)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "คุณเป็นผู้ช่วยที่สามารถใช้เครื่องมือในการคำนวณ, ตอบคำถามเกี่ยวกับฐานข้อมูลลูกค้า และค้นหาข้อมูลจาก Wikipedia ได้ กรุณาตอบเป็นภาษาไทยอย่างกระชับเมื่อสรุปผลลัพธ์ และใช้ประวัติการสนทนาเพื่อทำความเข้าใจคำถามและบริบท"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# สร้าง Agent ตัวหลักที่ใช้ Tools list และ Prompt ที่ปรับปรุงแล้ว
main_agent = create_tool_calling_agent(llm2, all_tools, prompt)

In [ ]:
# สร้าง Agent Executor สำหรับ Agent ตัวหลัก
main_agent_executor = AgentExecutor(agent=main_agent, tools=all_tools, verbose=True, handle_parsing_errors=True) # อาจจะเพิ่ม handle_parsing_errors=True ที่นี่ด้วยหากจำเป็น

In [ ]:
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# สร้าง RunnableWithMessageHistory โดยใช้ main_agent_executor
conversational_agent_with_history = RunnableWithMessageHistory(
    main_agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    handle_parsing_errors=True,
)

In [ ]:
session_id = "new-session"

In [ ]:
# คำถามแรก
result1 = conversational_agent_with_history.invoke(
    {"input": "มีลูกค้าทั้งหมดกี่คนในฐานข้อมูล?"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบแรก:", result1)

In [ ]:
# คำถามที่สอง (อ้างอิงถึงคำถามแรก)
result2 = conversational_agent_with_history.invoke(
    {"input": "พวกเขาอยู่ในประเทศอะไรบ้าง?"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบที่สอง:", result2)

In [ ]:
# ลองคำถามที่ต้องใช้ Calculator Tool ด้วย
result3 = conversational_agent_with_history.invoke(
    {"input": "ผลรวมของ 123 กับ 456 คือเท่าไหร่?"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบที่สาม:", result3)

In [ ]:
# ทดสอบ Agent ที่มี Memory และ Tools (Calculator, SQL, Wikipedia)

session_id = "multi-tool-session" # ใช้ session_id ใหม่สำหรับการทดสอบนี้

In [ ]:
# คำถามที่ต้องใช้ SQL Tool
result_sql = conversational_agent_with_history.invoke(
    {"input": "มีลูกค้าทั้งหมดกี่คนในฐานข้อมูล?"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบ (SQL):", result_sql)

In [ ]:
# คำถามที่ต้องใช้ Calculator Tool
result_calc = conversational_agent_with_history.invoke(
    {"input": "ผลคูณของ 99 กับ 101 คือเท่าไหร่?"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบ (Calculator):", result_calc)

In [ ]:
# คำถามที่ต้องใช้ Wikipedia Tool
result_wiki = conversational_agent_with_history.invoke(
    {"input": "ประเทศไทยมีเมืองหลวงชื่ออะไร"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบ (Wikipedia):", result_wiki)

In [ ]:
# ลองถามคำถามต่อเนื่องที่ต้องใช้ Memory และ Tool
result_context = conversational_agent_with_history.invoke(
    {"input": "ประชากรของเมืองหลวงนั้นมีประมาณเท่าไหร่"},
    config={"configurable": {"session_id": session_id}}
)
print("คำตอบ (Context + Wikipedia):", result_context)

In [ ]:
path = '/content/drive/MyDrive/RAG/MCU'

with engine.begin() as conn:
    conn.exec_driver_sql(f"CREATE OR REPLACE TABLE movies AS SELECT * FROM '{path}/movies.csv'")

In [ ]:
with engine.begin() as conn:
    conn.exec_driver_sql(f"CREATE OR REPLACE TABLE characters AS SELECT * FROM '{path}/characters.csv'")

In [ ]:
with engine.begin() as conn:
    conn.exec_driver_sql(f"CREATE OR REPLACE TABLE mcu AS SELECT * FROM '{path}/mcu.csv'")

In [ ]:
tables = db.get_usable_table_names()
tables